# Dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.odr import ODR, Model, Data, RealData
from scipy.optimize import curve_fit
import seaborn as sns
import uncertainties as uc
from uncertainties.umath import sqrt
from scipy.optimize import fsolve
from scipy.interpolate import interp1d
from scipy.signal import correlate
from scipy.signal import find_peaks
from scipy.signal import savgol_filter
from scipy.fft import fft

# Formatting

In [2]:
plt.style.use('Solarize_Light2')   #Options
                                    #Solarize_Light2
                                    #_classic_test_patch
                                    #_mpl-gallery
                                    #_mpl-gallery-nogrid
                                    #bmh
                                    #classic
                                    #dark_background
                                    #fast
                                    #fivethirtyeight
                                    #ggplot
                                    #grayscale
                                    #seaborn-v0_8
                                    #seaborn-v0_8-bright
                                    #seaborn-v0_8-colorblind
                                    #seaborn-v0_8-dark
                                    #seaborn-v0_8-dark-palette
                                    #seaborn-v0_8-darkgrid
                                    #seaborn-v0_8-deep
                                    #seaborn-v0_8-muted
                                    #seaborn-v0_8-notebook
                                    #seaborn-v0_8-paper
                                    #seaborn-v0_8-pastel
                                    #seaborn-v0_8-poster
                                    #seaborn-v0_8-talk
                                    #seaborn-v0_8-ticks
                                    #seaborn-v0_8-white
                                    #seaborn-v0_8-whitegrid
                                    #tableau-colorblind10

plt.rcParams.update({

    'figure.dpi': 225,                 # Resolution of figures
    'font.size': 10,                   # Default font size
    'font.family': 'sans-serif',       # Font family
    'font.sans-serif': ['DejaVu Sans'],
    
    'axes.titlesize': 10,              # Title font size
    'axes.labelsize': 8,              # Axis label font size
    'axes.linewidth': 0.8,             # Width of the axis lines
    'axes.grid': True,                 # Show grid
    # 'grid.color': '#dddddd',
    'grid.linestyle': '--',            # Grid line style (dashed)
    'grid.linewidth': 0.5,             # Grid line width
    
    'xtick.labelsize': 8,             # X-axis tick label size
    'ytick.labelsize': 8,             # Y-axis tick label size
    'xtick.direction': 'in',           # X-axis tick direction
    'ytick.direction': 'in',           # Y-axis tick direction

    'lines.linewidth': 0.8,            # Line width for plots
     'lines.markersize': 0.3,             # Marker size for scatter plots
     'lines.marker': 'o',               # Default marker shape

    'legend.frameon': True,            # Frame around legend
    'legend.framealpha': 0.8,          # Transparency of legend frame
    'legend.fontsize': 9,             # Font size in legend
    'legend.loc': 'best',              # Default legend location
})


# Useful Functions

In [3]:
def rSquared(f_model, x, y, Params):
    y_pred = f_model(*Params, x)  
    ss_res = np.sum((y - y_pred) ** 2)  
    ss_tot = np.sum((y - np.mean(y)) ** 2)  
    return 1 - (ss_res / ss_tot)

def Residuals(y_measured,y_modelled):
    return y_measured - y_modelled

def MeanSquareError(y_measured,y_modelled):
    return ((1/len(y_measured))*(np.sum((y_measured-y_modelled)**2)))

def RootMeanSquareError(y_measured,y_modelled):
    return np.sqrt(MeanSquareError(y_measured,y_modelled))

def MeanAbsError(y_measured,y_modelled):
    return ((1/len(y_measured))*(np.sum(np.abs(y_measured-y_modelled))))

def ConfidenceInterval(y_measured,y_modelled,confInt=95):
    if(confInt == 95):
        Z = 1.96
    elif(confInt == 90):
        Z = 1.645
    elif(confInt == 99):
        Z = 2.576
    return Z*np.sqrt((1/len(y_measured))*(np.sum((y_measured-y_modelled)**2)))

def ArrayWithStaticUnc(Array,unc):
    a = []
    b = []
    for i in range(len(Array)):
            a.append(uc.ufloat(Array[i],unc))
            b.append(unc)
    return np.array(a), np.array(b)

def UnpackUncArray(Array):
     value = [Array[i].nominal_value for i in range(len(Array))]
     std = [Array[i].std_dev for i in range(len(Array))]
     return np.array(value),np.array(std) 

# Non Linear Curve Fiting Function
def NonLinAnalysis(f_model, x, y, ParamsArr, weights):
    if ParamsArr is None:
        Params = None
        std = None 
        R2 = rSquared(f_model,x,y,Params)
    else:
        Params, cov = curve_fit(f_model, x, y, p0=ParamsArr, sigma=weights)
        std = np.sqrt(np.diag(cov))
        R2 = rSquared(f_model,x,y,Params)
    return Params, std, R2
 
def RR2(y,y_pred):
   ss_res = np.sum((y - y_pred) ** 2)
   ss_tot = np.sum((y - np.mean(y)) ** 2)
   R2 = 1 - (ss_res / ss_tot)
   return R2

def sortByArray1(array1, array2):
    sorted_pairs = sorted(zip(array1, array2))
    sorted_array1, sorted_array2 = zip(*sorted_pairs)
    return list(sorted_array1), list(sorted_array2)

# Inputting and Formatting Data

In [4]:
Resonance = pd.read_csv("Data/ResonantFreq.csv")
DistanceAmp = pd.read_csv("Data/DistanceVAmp.csv")
DetWavelength = pd.read_csv("Data/DetermineWavelength.csv")
TransistTime = pd.read_csv("Data/TransitTimeMethod.csv")

# Task I

In [ ]:
DistanceAmp

In [6]:
def mitUncR(R):
    return 0.001*R
def mitUncV(V):
    return 0.005*V
def mitUnctime(t):
    return 0.01*t

In [ ]:
distance = [uc.ufloat(val, mitUncR(val)) for val in DistanceAmp["distance(cm)"]]
voltage = [uc.ufloat(val, mitUncV(val)) for val in DistanceAmp["voltage(V)"]]


def inverse_square_law(d, A, C):
    return A / (d + C)**2 

popt, pcov = curve_fit(inverse_square_law, [d.nominal_value for d in distance], [v.nominal_value for v in voltage])
A_fit, C_fit = popt

voltage_fit = inverse_square_law(np.array([d.nominal_value for d in distance]), A_fit, C_fit)
R2 = RR2([v.nominal_value for v in voltage], voltage_fit)

plt.figure(figsize=(8, 6))
plt.errorbar(DistanceAmp["distance(cm)"], [v.nominal_value for v in voltage], 
             yerr=[v.std_dev for v in voltage], 
             xerr=[d.std_dev for d in distance], 
             fmt="o", markersize=5, capsize=5, elinewidth=1, color="darkblue", label="Measured Data")
plt.plot(DistanceAmp["distance(cm)"], voltage_fit, color="green", linewidth=2, linestyle="--", label=f"Curve fit line")
print(A_fit, C_fit)

Table1 = [
        [r"$r^2$", f"{R2:.5g}"],
        ]
plt.table(cellText= Table1,
          colLabels=["","Modelled"],
          loc='center right',
          cellLoc='left',
          colWidths=[0.1,0.12],
          fontsize=4, zorder=100)

plt.xlabel("Distance (cm)", fontsize=12)
plt.ylabel("Voltage (V)", fontsize=12)
plt.title("The ultrasound signal as a function of distance", fontsize=14)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.legend(fontsize=10)
plt.grid(color="black", linestyle="--", linewidth=0.5)
plt.tight_layout()

plt.show()


In [ ]:
Resonance

In [ ]:
x = Resonance["Freq(KHz)"]
y = Resonance["Volt(V)"]
plt.errorbar(x, y, markersize=5, color='darkred', fmt="o")
plt.xlabel("Frequency(kHz)", fontsize=12)
plt.ylabel("Voltage (V)", fontsize=12)
plt.title("Resonance Frequency", fontsize=14)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.legend(fontsize=10)
plt.grid(color="black", linestyle="--", linewidth=0.5)
plt.tight_layout()
plt.show()

In [ ]:
DetWavelength["FrequencymitUnc"] = [uc.ufloat(DetWavelength["frequency(kHz)"][i],4e-9) for i in range(len(DetWavelength))]
DetWavelength["DistancemitUnc"] = [uc.ufloat(DetWavelength["distance(cm)"][i],0.01) for i in range(len(DetWavelength))]
Wavelength = [DetWavelength["DistancemitUnc"][i+1] - DetWavelength["DistancemitUnc"][i] for i in range(len(DetWavelength)-1)]
WavelengthAvg = np.mean(Wavelength)
Velo = WavelengthAvg*(DetWavelength["FrequencymitUnc"][0])*10
print(f"Wavelength: {WavelengthAvg} m")
print(f"Speed of Sound from Lissajous Figures Method \n{Velo} m/s")

In [ ]:
distance1 = [uc.ufloat(val, mitUncR(val)) for val in TransistTime["distance(cm)"]]
time = [uc.ufloat(val, mitUnctime(val)) for val in TransistTime["deltaTime(us)"]]

plt.figure(figsize=(8, 6))
plt.errorbar([t.nominal_value for t in time], [d.nominal_value for d in distance1],
             yerr=[t.std_dev for t in time], 
             xerr=[d.std_dev for d in distance1], 
             fmt="o", markersize=5, capsize=5, elinewidth=1, color="darkblue", label="Measured Data")

plt.xlabel("Time (ms)", fontsize=12)
plt.ylabel("Distance (cm)", fontsize=12)
plt.title("Time Transient Method", fontsize=14)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.legend(fontsize=10)
plt.grid(color="black", linestyle="--", linewidth=0.5)
plt.tight_layout()
plt.show()

In [ ]:
TransistTime

In [ ]:
TransistTime["DistancemitUnc"] = [uc.ufloat(TransistTime["distance(cm)"][i]*1e-2,0.01*1e-2) for i in range(len(TransistTime))]
TransistTime["TimemitUnc"] = [uc.ufloat(TransistTime["deltaTime(us)"][i]*1e-6,0.1*1e-6) for i in range(len(TransistTime))]
TransitVelocities = [TransistTime["DistancemitUnc"][i]/TransistTime["TimemitUnc"][i] for i in range(len(TransistTime))]
AvgTransitVelocities = np.mean(TransitVelocities)
print(f"Speed of Sound from Time Transit Method \n{AvgTransitVelocities} m/s")

In [ ]:
def gamma(V):
    return (V**2 * 1.225)/101325

print(f"Adiabatic Constant from Lissajour Figure Method \n{gamma(Velo)}")
print(f"Adiabatic Constant from Time Transit Method \n{gamma(AvgTransitVelocities)}")


# Task II

In [ ]:
ResFreq = uc.ufloat(23.8, 0.2)
def gammaResTube(ResFreq):
    return 297.1*(ResFreq**2/101325)
print(f"Adiabatic Constant from Resonance-Tube Method \n{gammaResTube(ResFreq)}")


In [ ]:
TheoreticalAdConst = 1.4
g= 190
a = [200-g,300-g,400-g]
b=[gammaResTube(ResFreq).nominal_value,gamma(AvgTransitVelocities).nominal_value,gamma(Velo).nominal_value]
plt.figure(figsize=(8,6),dpi=200)
plt.ticklabel_format(axis='y', style='sci', scilimits=(-2, 2))
plt.xticks(ticks=[-90,200-g,300-g,400-g],labels=['Theo',"ResTube",'TimeTrans',"UltraSound"])
plt.xlim(-200,500-g)
plt.scatter(-90,1.4,s=10,color="cyan",edgecolors='k',linewidth=0.5,zorder=10,label="Theoretical")
plt.errorbar(a, b, yerr=[gammaResTube(ResFreq).std_dev, gamma(AvgTransitVelocities).std_dev, gamma(Velo).std_dev], fmt='o', color="orange", ecolor="k", elinewidth=0.5, capsize=3, zorder=10, label="Measured", markeredgecolor="k", markersize=3, 
linewidth=0.5)
#plt.scatter(a,c,s=10,color="#e75480",edgecolors='k',linewidth=0.5,zorder=10,label="Theoretical")
#plt.errorbar(a,c,yerr=cerr,fmt="o",capsize=2,color="#e75480")
plt.legend()
plt.ylabel(r'$\gamma$')
plt.title("Adiabatic Constants Comparision")
plt.show()


In [ ]:
print(gammaResTube(ResFreq))
print(gamma(AvgTransitVelocities))
print(gamma(Velo))